In [ ]:
import tensorflow as tf  # Importar TensorFlow

# Imprimir la versión de TensorFlow instalada
print(tf.__version__)

import sklearn.metrics  # Importar métricas de Scikit-learn

# Imprimir la versión de Scikit-learn instalada
print(sklearn.__version__)

In [ ]:
# Instalación de dependencias necesarias
!pip install vit-keras tensorflow-addons opencv-python matplotlib

In [ ]:
# Bibliotecas estándar de Python
import os  # Manejo de rutas y directorios
import random  # Generación de valores aleatorios
import math  # Funciones matemáticas
import gc  # Recolección manual de basura para liberar memoria
from datetime import datetime  # Manejo de fechas y horas

# Bibliotecas de terceros
import numpy as np  # Operaciones numéricas y manejo de matrices
import matplotlib.pyplot as plt  # Visualización de gráficos y datos
import cv2  # OpenCV para procesamiento de imágenes

# Vision Transformer (ViT) para clasificación de imágenes
from vit_keras import vit

# Imprimir información sobre la versión de TensorFlow y los dispositivos disponibles
print(tf.__version__)
print("Dispositivos físicos disponibles:", tf.config.list_physical_devices())
print("Dispositivos GPU disponibles:", tf.config.list_physical_devices('GPU'))


In [ ]:
from tensorflow.keras.models import load_model  # Función para cargar un modelo preentrenado

# Cargar el modelo guardado desde un archivo .keras
model = load_model(
    "/Users/marcperezg/Desktop/Uni/TFG/Melanoma/2_optuna_checkpoint_model_v4.3.keras",
    safe_mode=False  # Se desactiva el modo seguro para permitir carga sin restricciones
)


In [ ]:
# Tamaño de la imagen y batch size
IMG_SIZE = 224  # Tamaño de las imágenes de entrada (debe coincidir con el modelo)
BATCH_SIZE = 16  # Tamaño del batch para la evaluación

# Directorio de las imágenes de prueba
test_dir = '/Users/marcperezg/Desktop/Uni/TFG/Melanoma/test'

# Crear dataset de prueba desde el directorio
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',  # Detecta automáticamente las subcarpetas como etiquetas (ej: benign, malignant)
    label_mode='int',  # Etiquetas en formato entero (0 para benign, 1 para malignant)
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE)  # Redimensiona las imágenes al tamaño esperado por el modelo
)

# Función para normalizar las imágenes
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Escala los valores de píxeles de 0 a 1
    return image, label

# Aplicar normalización al dataset de prueba
test_dataset = test_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

# Optimización del rendimiento en la carga de datos
AUTOTUNE = tf.data.AUTOTUNE
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score, confusion_matrix, roc_auc_score

# Función para evaluar el modelo en el conjunto de prueba
def evaluate_model(model, test_dataset):
    # Obtener etiquetas verdaderas y predicciones de probabilidad
    y_true = np.concatenate([labels.numpy() for _, labels in test_dataset])
    y_pred_proba = model.predict(test_dataset)  # Predicciones de probabilidad para todo el dataset

    # Convertir probabilidades en clases binarias
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    # Calcular métricas de evaluación
    f1 = f1_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)  # Sensibilidad (Recall)
    accuracy = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred_proba)  # AUC-ROC

    # Calcular la especificidad
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    specificity = tn / (tn + fp)

    # Mostrar resultados de evaluación
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensibilidad (Recall): {sensitivity:.4f}")
    print(f"Especificidad: {specificity:.4f}")
    print(f"Precisión (Accuracy): {accuracy:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print(f"Matriz de Confusión:\n{conf_matrix}")
    print(f"Malignos clasificados correctamente (TP): {tp}")
    print(f"Malignos clasificados incorrectamente (FN): {fn}")
    print(f"Benignos clasificados correctamente (TN): {tn}")
    print(f"Benignos clasificados incorrectamente (FP): {fp}")

    # Devolver métricas en un diccionario
    return {
        "F1 Score": f1,
        "Sensitivity (Recall)": sensitivity,
        "Specificity": specificity,
        "Accuracy": accuracy,
        "AUC-ROC": auc_roc,
        "Confusion Matrix": conf_matrix
    }

# Evaluar el modelo con el conjunto de prueba
metrics = evaluate_model(model, test_dataset)
